# Der erwartungswert im allgemeinen Fall für das Wort $W$


In [ ]:
import numpy as np
import pandas as pd 
import copy
import itertools as it
from itertools import product as carprod
import scipy
from scipy.special import comb
import pprint
from scipy.special import factorial
from numpy import product

In [234]:
def without_max(A):
    A_ref=copy.copy(A)
    for i in range(len(A)):
       if i ==np.argmax(A):
            A_ref[i]=0
    return(A_ref)

In [235]:
def rounded_halves(A):
    X=[]
    for i in range(len(A)):
        X.append(list(range(int(np.floor(A[i]/2)+1))))
    return(X)

# Berechne die Menge der perfekten Matchings vom vollständigen k-partiten Graphen $\mathbf{P}^2_{\mathbb K_{W}}$
 

In [237]:
def all_sum_rec(target, current_sum, start, output, result):
 if current_sum == target:
   output.append(copy.copy(result))

 for i in range(start, target):
   temp_sum = current_sum + i
   if temp_sum <= target:
     result.append(i)
     all_sum_rec(target, temp_sum, i, output, result)
     result.pop()
   else:
     return
def all_sum(target,leng):
 output = []
 result = []
 all_sum_rec(target, 0, 1, output, result)
 output1 =np.zeros((len(output),leng),dtype=int)
 for i in range(len(output)):
        a=np.array(output[i])
        if len(a) <= leng:
           for j in range(len(a)):
              output1[i,j]=a[j]
 Temp_Matching=np.zeros((0,),dtype=int)
 for i in range(len(output1)):
    if sum(output1[i])!=0 and (np.sum(output1[i])-2*np.max(output1[i]))>=0:
          Temp_Matching = np.concatenate([Temp_Matching,output1[i]],axis=0)
 Temp_Matching=np.reshape(Temp_Matching,(int(len(Temp_Matching)/leng),leng))
 return Temp_Matching

def Matching_with_and_without_max(W):
    D=np.sum(W)-(2*np.max(W))
    if D>0:
        A=all_sum(D,len(W)-1)
        Matching_without_max=np.zeros((0,len(W)),dtype=int)
        for x in A:
            M=list(it.permutations(x))
            for m in M:
                v=np.array(m)
                v=np.insert(v,np.argmax(W),0,axis=0)
                if (v<=W).all():
                    Matching_without_max=np.concatenate((Matching_without_max,v.reshape(1,len(W))),axis=0)
    elif D==0:
        Matching_without_max=np.zeros((1,len(W)),dtype=int)
    Matching_without_max=np.unique(Matching_without_max,axis=0)
    Matching_with_max=np.zeros([0,len(W)],dtype=int)
    for x in Matching_without_max:
        b=W-x
        Matching_with_max=np.concatenate([Matching_with_max,b.reshape(1,len(W))],axis=0)
    M1=np.zeros((0,len(W)),dtype=int)
    M=[]
    for i in range(len(Matching_with_max)):
        M1=np.concatenate((Matching_with_max[i].reshape(1,len(W)),Matching_without_max[i].reshape(1,len(W))),axis=0)
        M.append(M1)
    return(M)

def condition(K):
    index=True
    for x in K:
        if np.sum(x[-1])-(2*np.max(x[-1]))!=0:
            index=False
            break
    return(index)

def extension_of_matching(L):
    K=[]
    for x in L:
        if np.sum(x[-1])-(2*np.max(x[-1]))!=0:
            z=Matching_with_and_without_max(x[-1])
            for y in z:
                G=np.zeros((0,len(y[0])),dtype=int)
                G=np.concatenate((x[:-1],y[0].reshape(1,len(y[0])),y[1].reshape(1,len(y[0]))),axis=0)
                K.append(G)
        else:
            K.append(x)
    return(K)

def find_all_matching(W):
    secondstep=Matching_with_and_without_max(W)
    index=condition(secondstep)
    while index==False:
          secondstep=extension_of_matching(secondstep)
          index=condition(secondstep)
    return secondstep

# Für jedes Matching aus der Menge $\mathbf{P}^2_{\mathbb K_{W}}$ berechne dessen Frequenz 
## \begin{equation}
    \mathfrak{fq}_{p,\mathbb K_{W}}:=\frac{W!}{p_{12}!p_{13}!\dots p_{1k}!p_{23}!\dots p_{(k-1)k}!}
\end{equation} 

In [238]:
def frequency_of_Matching(All_Matching,L):
    frequency_of_Matching=np.zeros((0,1))
    for x in All_Matching:
        fr=np.ones((1,1))
        for y in x:
            F=without_max(y)
            fr*=product(factorial(F))
        frequency_of_Matching=np.concatenate((frequency_of_Matching,L/fr))
    return(frequency_of_Matching)


## Für ein Wort $W$,  berechne das Bild jedes Matchings aus der Menge $\mathbf P^2_{\mathbb K_W}$  durch die Funktion $\mathbf h$
### \begin{align*}
    &p=(12)^{p_{12}}(13)^{p_{13}}\dots (1k)^{p_{1k}}(23)^p_{23}\dots (2k)^{p_{2k}}\dots (k-1k)^{p_{k-1k}}\\
    %
    &\mathbf h(p)=[+{\sum_{\ell>1}^kp_{1\ell}}][-{p_{12}}+{\sum_{\ell>2}^kp_{2\ell}}]\dots[-{\sum_{\ell=1}^{l-1}p_{\ell l}}+{\sum_{\ell>l}^kp_{l\ell}}]\dots[-{\sum_{\ell=1}^{k-1}p_{\ell k}}]
\end{align*}

In [464]:
def partialDyckWords(M):
    DyckWords=np.zeros(M.shape,dtype=int)
    k=np.argmax(M)
    for j in range(M.shape[0]):
         DyckWords[j]=int(M[j]*np.sign(k-j))
    DyckWords[k]=int(-np.sum(DyckWords))
    return(DyckWords)

def DYCKWORDS(Matching,W):
    DW=np.zeros((0,len(W)),dtype=int)
    for i in range(len(Matching)):
        DW_part=0
        for j in range(len(Matching[i])):
            DW_part=DW_part+partialDyckWords(Matching[i][j])
        DW=np.concatenate((DW,DW_part.reshape(1,len(Matching[0][0]))),axis=0)
    return(DW)

# Berechne die Frequenz $\mathfrak{fq}_{(\mathfrak{dw},W)}$einer Dyckwort-Projektion in einem Graphen $\mathbb k_{W}$ 

In [465]:
def FQ_dw_W(DW,frequency,Unique):
    Frequency_of_dw_in_W=np.zeros((0,1))
    for x in  Unique:
        frequency_of_dw_in_W=np.zeros((1,1))
        for i in range(len(DW)):
            if (x==DW[i]).all():
                frequency_of_dw_in_W=frequency_of_dw_in_W+frequency[i]
        Frequency_of_dw_in_W=np.concatenate((Frequency_of_dw_in_W,frequency_of_dw_in_W),axis=0)    
    return(Frequency_of_dw_in_W)

In [466]:
def projektions_of_Dyckwords_with_frequeny_on(W):
    L=product(factorial(W))
    All_Matching=find_all_matching(W)
    frequency=frequency_of_Matching(All_Matching,L)
    DW=DYCKWORDS(All_Matching,W)
    Unique_DW=np.unique(DW,axis=0)
    Frequency_of_dw_in_W=FQ_dw_W(DW,frequency,Unique_DW)
    return(Unique_DW,Frequency_of_dw_in_W)

## \begin{align*}
    \mathcal P(\varphi,\mathfrak{dw}):=\sum_{w_{\mathfrak{dw}} \in \mathfrak W_{\mathfrak{dw}}}\frac{\mathfrak{fq}_{(\mathfrak{dw},w_{\mathfrak{dw}})}}{\frac{W-w_{\mathfrak{dw}}}{2}! w_{\mathfrak{dw}}!}(\frac{- (1-\varphi)}{ \varphi})^{\frac{w_{\mathfrak{dw}}}{2}}
\end{align*}
## \begin{equation*}
    \operatorname{E}[\langle\operatorname{ISS}(X),W\rangle]=W!(\frac{\sigma^2}{1+\varphi})^{\frac{W}{2}}\sum_{\mathfrak{dw} \in \mathfrak{DW}}P(\varphi,\mathfrak{dw})\langle \operatorname{ISS}(Z),\mathfrak{dw} \rangle
\end{equation*}
### Im Code \begin{align*} 
x1=\frac{W-w_{\mathfrak{dw}}}{2} \quad  w=w_{\mathfrak{dw}} \quad power=\frac{w_{\mathfrak{dw}}}{2} 
\end{align*}

In [510]:
def All_Dyck_Words(W):
    X=rounded_halves(W)
    Graphs = [i for i in it.product(*X)]
    ALL_DYCK_WORDS=[]
    for x in Graphs:
        x1=np.array(x)
        w=W-2*x1
        p=np.array([int(np.sum(w)/2)])
        D=np.sum(w)-(2*np.max(w))
        if D>=0:
            [Unique_DW,Frequency_of_dw_in_W]=projektions_of_Dyckwords_with_frequeny_on(w)
            x1_factorial=product(factorial(x1))
            w_factorial=product(factorial(w))
            coefficient=Frequency_of_dw_in_W/(x1_factorial*w_factorial)
            ALL_DYCK_WORDS.append([p,coefficient,Unique_DW])
    return(ALL_DYCK_WORDS)

In [489]:
def the_expected_value_of_word(W):
    All_Dyck_Words_withcoefficient=All_Dyck_Words(W)
    the_set_of_all_projektion=np.zeros((0,len(W)))
    for i in range(len(All_Dyck_Words_withcoefficient)):
        the_set_of_all_projektion=np.concatenate((the_set_of_all_projektion,All_Dyck_Words_withcoefficient[i][2]),axis=0)
    unique_the_set_of_all_projektion=np.unique(the_set_of_all_projektion,axis=0)
    the_expected_value_of_word_W=[]
    for x in unique_the_set_of_all_projektion:
        power_and_coefficent_polynomial=[]
        for y in All_Dyck_Words_withcoefficient:
            for j in range(len(y[2])):
                if (x==y[2][j]).all():
                    power_and_coefficent_polynomial.append((fractions.Fraction(y[1][j][0]).limit_denominator(),y[0][0]))
        the_expected_value_of_word_W.append([power_and_coefficent_polynomial,x.astype(int)])
    return(the_expected_value_of_word_W)

In [485]:
W=np.array([1,2,3,4,5,6,7])
the_expected_value_of_word_W=the_expected_value_of_word(W)

In [486]:
Df_the_expected_value_of_word_W=pd.DataFrame(the_expected_value_of_word_W,columns=['(fracture,Power_of_the_constant)','Dyckword'])

In [487]:
Df_the_expected_value_of_word_W

,"(fracture,Power_of_the_constant)",Dyckword
0,"[(1/48, 4), (1/144, 3), (1/48, 3), (1/144, 2)]","[1, 0, -1, 0, 1, 0, -1]"
1,"[(1/48, 6), (1/32, 5), (1/48, 5), (1/32, 4)]","[1, 0, -1, 0, 1, 2, -3]"
2,"[(1/480, 8), (1/96, 7), (1/480, 7), (1/96, 6)]","[1, 0, -1, 0, 1, 4, -5]"
3,"[(1/2880, 9), (1/2880, 8)]","[1, 0, -1, 0, 1, 6, -7]"
4,"[(1/72, 6), (1/48, 5), (1/72, 5), (1/48, 4)]","[1, 0, -1, 0, 3, -2, -1]"
...,...,...
485,"[(105/4, 13), (105/4, 12), (315/16, 12), (315/...","[1, 2, 3, 4, -1, -4, -5]"
486,"[(105/8, 14), (35/2, 13), (15/4, 12), (315/32,...","[1, 2, 3, 4, -1, -2, -7]"
487,"[(385/32, 13), (385/48, 12), (77/96, 11)]","[1, 2, 3, 4, 1, -6, -5]"
488,"[(385/32, 14), (275/32, 13), (385/48, 13), (27...","[1, 2, 3, 4, 1, -4, -7]"


In [512]:
A=np.array([1,2,3,4])
the_expected_value_of_word(A)

[[[(Fraction(1, 2), 2), (Fraction(1, 2), 1)], array([ 1,  0, -1,  0])],
 [[(Fraction(1, 2), 4),
   (Fraction(1, 1), 3),
   (Fraction(1, 2), 3),
   (Fraction(1, 1), 2)],
  array([ 1,  0,  1, -2])],
 [[(Fraction(1, 6), 5), (Fraction(1, 6), 4)], array([ 1,  0,  3, -4])],
 [[(Fraction(1, 4), 3)], array([ 1,  2, -3,  0])],
 [[(Fraction(3, 2), 4), (Fraction(3, 2), 3)], array([ 1,  2, -1, -2])],
 [[(Fraction(3, 4), 5), (Fraction(1, 2), 4)], array([ 1,  2,  1, -4])]]